In [36]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import re


def readtext(z):
    text=pd.read_csv(z+'Localizations/english.txt',quotechar='#',skipinitialspace=True)
    text1=pd.read_csv(z+'Localizations/russian.txt',quotechar='#',skipinitialspace=True)
    text2=pd.read_csv(z+'Localizations/br.txt',quotechar='#',skipinitialspace=True)
    text3=pd.read_csv(z+'Localizations/kr.txt',quotechar='#',skipinitialspace=True)
    t1=text2[['id','text_br']]
    text=text.join(t1.set_index('id'),on='id')
    t1=text3[['id','text_kr']]
    text=text.join(t1.set_index('id'),on='id')
    
    
    text['text_ru']=text1['text_ru']
    text['item']=text['id']*0.
    text['order']=text['id']*0.
    text['parent']=text['id']*0.
    text['destination']=text['id']*0.
    text['type']=text['text_en']
    text['type']=''
    text['authorid']=text['id']*0.
    text['name_en']=text['type']
    text['name_ru']=text['type']
    text['name_br']=text['type']
    text['name_kr']=text['type']
    text['authorname']=text['type']
    return text

def readlines(z,text):
    file1 = open(z+'GameString.xml', 'r') 
    Lines = file1.readlines()
    for line in Lines:
        l=(re.split('<|>|"',line))
        if (l[1]=='Item id='):
            item=l[2]
        if (l[1]=='Parent'):
            parent=l[2]
            
            a=np.where(int(item)==text['id'])[0]
            text['parent'][a]=parent
            text['type'][a]='string'
            ids="0"
    
    file1 = open(z+'Speech.xml', 'r') 
    Lines = file1.readlines() 
    
    for line in Lines:
        l=(re.split('<|>|"',line))
        if (l[1]=='Item id='):
            item=l[2]
        if (l[1]=='Text'):
            ids=l[2]
        if (l[1]=='OrderIndex'):
            order=l[2]
        if (l[1]=='AuthorGuid'):
            author=l[2]
        if (l[1]=='Name'):
            name=l[2]
        if (l[1]=='Parent'):
            parent=l[2]
            
            a=np.where(int(ids)==text['id'])[0]
            text['item'][a]=item
            #text['order'][a]=order
            text['parent'][a]=parent
            text['type'][a]='character'
            text['authorid'][a]=author
            text['authorname'][a]=name
            ids="0"
        
    file1 = open(z+'Reply.xml', 'r') 
    Lines = file1.readlines() 
    
    for line in Lines:
        l=(re.split('<|>|"',line))
        if (l[1]=='Item id='):
            item=l[2]
        if (l[1]=='Text'):
            ids=l[2]
        if (l[1]=='OrderIndex'):
            order=l[2]
        if (l[1]=='Parent'):
            parent=l[2]
            
            a=np.where(int(ids)==text['id'])[0]
            text['item'][a]=item
            text['order'][a]=order
            text['parent'][a]=parent
            text['type'][a]='player'
            ids="0"
        
            
    a=np.where(text['type']=='character')[0]
    text['destination'][a]=text['item'][a]


    return text

def graphlink(z,text):

    file1 = open(z+'GraphLink.xml', 'r') 
    Lines = file1.readlines()
    source="0"
    s="  "
    d="  "
    n=-1
    for line in Lines:
        l=(re.split('<|>|"',line))
        if (l[1]=='Source'):
            source=l[2]
            s=l[2][0:2]
        if (l[1]=='SourceExitPointIndex'):
            order=l[2]
        if (l[1]=='Destination'):
            destination=l[2]
            d=l[2][0:2]
            if str(d)!='33':
                a=np.where((text['type']=='player') & (text['parent']==source) & (text['order']==order))[0]
                text['destination'][a]=destination
            if (str(s)=='33'):
                b=np.where(text['item']==l[2])[0]
                text['parent'][b]=n
                n=n-1
                
    return text

def setname(text,name):
    a=np.where(text['type']=='player')[0]
    text['name_en'][a]=name[0]
    text['name_ru'][a]=name[1]
    text['name_br'][a]=name[2]
    text['name_kr'][a]=name[3]
    
    a=np.where(text['type']=='character')[0]
    for i in a:
        b=np.where(text['authorid'][i]==text['parent'])[0]
        try:
            text['name_ru'][i]=text['text_ru'][b].values[0]
            text['name_en'][i]=text['text_en'][b].values[0]
            text['name_br'][i]=text['text_br'][b].values[0]
            text['name_kr'][i]=text['text_kr'][b].values[0]
        except:
            pass
    return text

def talking(z,text):
    file1 = open(z+'Talking.xml', 'r') 
    Lines = file1.readlines()
    i=0
    n=-1
    for line in Lines:
        l=(re.split('<|>|"',line))
        if (l[1]=='Item id='):
            i=0
        if i==3:
            a=np.where(text['item']==l[2])[0]
            text['parent'][a]=n
            n=n-1
        i=i+1
    return text

def fixbroken(text):
    
    a=np.where(text['id']==6192355001845370)[0]
    text['destination'][a]=0
    a=np.where(text['id']==6192355001499069)[0]
    text['parent'][a]=-text['parent'][a]
    
    start=np.where((text['type']=='character') & (text['name_ru'].isnull()))[0]
    text['name_en'].iloc[start]='Person'
    text['name_ru'].iloc[start]='Человек'
    text['name_br'].iloc[start]='Pessoa'
    text['name_kr'].iloc[start]='사람'
    start=np.where((text['type']=='character') &  (text['name_en'].notnull()) & (text['name_br'].isnull()))[0]
    for i in range(len(start)):
        a=np.where(text['text_en']==text['name_en'][start[i]])[0]
        b=np.where(text['text_br'].iloc[a].notnull())[0]
        if len(b)>0:
            text['name_br'].iloc[start[i]]=text['text_br'].iloc[a[b[0]]]
        b=np.where(text['text_kr'].iloc[a].notnull())[0]
        if len(b)>0:
            text['name_kr'].iloc[start[i]]=text['text_kr'].iloc[a[b[0]]]
    start=np.where((text['type']=='character') & (text['name_br'].isnull()))[0]
    text['name_br'].iloc[start]='Pessoa'
    start=np.where((text['type']=='character') & (text['name_kr'].isnull()))[0]
    text['name_kr'].iloc[start]='사람'
    start=np.where((text['type']=='character') & (text['name_en'].isnull()))[0]
    text['name_en'].iloc[start]='Person'
    
    a=np.where(text['id'].isnull()==False)[0]
    text=text.iloc[a]
    text['id']=text['id'].astype(int)
    text['id']=(text['id']-np.min(text['id']))+5
    
    rn=np.where((text['type']=='character') & (text['name_en']=='Person'))[0]
    text['name_en'].iloc[rn]=text['authorname'].iloc[rn]
    text['name_ru'].iloc[rn]=text['authorname'].iloc[rn]
    text['name_br'].iloc[rn]=text['authorname'].iloc[rn]
    text['name_kr'].iloc[rn]=text['authorname'].iloc[rn]

    return text

In [37]:
haru=['Haruspex','Гаруспик','Arúspice','창점술사']
bach=['Bachelor','Баклавр','Bacharel','학사']

zz=['PathologicSandbox/','MarbleNest/','PathologicPlagueIntro/','PathologicHaruspexIntro/']
names=[haru,bach,haru,haru]

for i in range(4):
    z=zz[i]
    name=names[i]
    
    text=readtext(z)
    text=readlines(z,text)
    text=graphlink(z,text)
    text=setname(text,name)
    #text=talking(z,text)
    text=fixbroken(text)
    
    text.to_csv(z.split('/')[0]+'.csv')

/Users/mkounkel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [38]:
def loop(a4,n,level,f,inarr,textarr,language='en'):
    
    q=np.unique(data['destination'][a4])
    if ((len(q)==1) & (len(a4)>1)):
        f.write('<div data-role="collapsible" class="collapsible" data-collapsed="true">')
        if language=='en': rep=' replies'
        if language=='ru': rep=' реплики'
        if language=='br': rep=' respostas'
        if language=='kr': rep=' 답글'
        f.write('<h1>'+str(level+1)+'.'+data['name_'+language][a4[0]]+', '+str(len(a4))+rep+'</h1>')
        for j4 in a4:
            f.write('<p class="'+str(data['id'][j4])+'">'+str(level+1)+'.'+'</p>')
            a=np.where(inarr==data['id'][j4])[0]
            if len(a)==0:
                inarr.append(data['id'][j4])
                textarr.append(data['name_'+language][j4]+': '+data['text_'+language][j4])
        
                
        n=n+1
        if data['destination'][j4]!=0:
            b4=np.where((data['item'] == data['destination'][j4]) & (data['type']=='character'))[0][0]
            a5=np.where((data['parent'] == data['item'][b4]) & (data['type']=='player'))[0]
            try:
                if data['id'][b4] in inarr:
                    f.write('<div data-role="collapsible" data-collapsed="true">')
                else:
                    f.write('<div data-role="collapsible" class="collapsible" data-collapsed="true">')
                a=np.where(inarr==data['id'][b4])[0]
                if len(a)==0:
                    inarr.append(data['id'][b4])
                    textarr.append(data['name_'+language][b4]+': '+data['text_'+language][b4])
                
                f.write('<h1 class="'+str(data['id'][b4])+'">'+str(level+2)+'.'+'</h1>')
                f.write('<p class="'+str(data['id'][b4])+'">'+str(level+2)+'.'+'</p>')
                
            except:
                print(level,data['name_'+language][b4],data['id'][b4],data['text_'+language][b4])
            n=n+1
            n=loop(a5,n,level+2,f,inarr,textarr,language=language)
            f.write('</div>')
        f.write('</div>')  
    
    else:
        for j4 in a4:
            try:
                if data['id'][j4] in inarr:
                    f.write('<div data-role="collapsible" data-collapsed="true">')
                else:
                    f.write('<div data-role="collapsible" class="collapsible" data-collapsed="true">')
                a=np.where(inarr==data['id'][j4])[0]
                if len(a)==0:
                    inarr.append(data['id'][j4])
                    textarr.append(data['name_'+language][j4]+': '+data['text_'+language][j4])
                
                f.write('<h1 class="'+str(data['id'][j4])+'">'+str(level+1)+'.'+'</h1>')
                f.write('<p class="'+str(data['id'][j4])+'">'+str(level+1)+'.'+'</p>')
                #print(str(data['text_en'][j4]))
                
            except:
                print(level,data['name_'+language][j4],data['id'][j4],data['text_'+language][j4])
                
                
            n=n+1
            if data['destination'][j4]!=0:
                b4=np.where((data['item'] == data['destination'][j4]) & (data['type']=='character'))[0][0]
                a5=np.where((data['parent'] == data['item'][b4]) & (data['type']=='player'))[0]
                try:
                    if data['id'][b4] in inarr:
                        f.write('<div data-role="collapsible" data-collapsed="true">')
                    else:
                        f.write('<div data-role="collapsible" class="collapsible" data-collapsed="true">')
                    a=np.where(inarr==data['id'][b4])[0]
                    if len(a)==0:
                        inarr.append(data['id'][b4])
                        textarr.append(data['name_'+language][b4]+': '+data['text_'+language][b4])
                    
                    f.write('<h1 class="'+str(data['id'][b4])+'">'+str(level+2)+'.'+'</h1>')
                    f.write('<p class="'+str(data['id'][b4])+'">'+str(level+2)+'.'+'</p>')
                except:
                    print(level,data['name_'+language][b4],data['id'][b4],data['text_'+language][b4])
                n=n+1
                n=loop(a5,n,level+2,f,inarr,textarr,language=language)
                f.write('</div>')
            f.write('</div>')    
    return n

In [39]:
def main(start,name='PathologicSandbox',header="Pathologic",language='en'):
    uniqs=np.unique(data['name_'+language][start])
    
    if language=='en':
        bt1,bt2='Expand all unique','Collapse All'
    if language=='ru':
        bt1,bt2='Развернуть все уникальные','Свернуть все'
    if language=='br':
        bt1,bt2='Expandir tudo','Recolher tudo'
    if language=='kr':
        bt1,bt2='고유한 항목 모두 확장','모든 축소'

    if name=='???':
        f = open('../html2_'+language+'/'+''.join('unknown')+'.html', "w")
    else:
        f = open('../html2_'+language+'/'+''.join(name.split(' '))+'.html', "w")
    
    f.write("""<!DOCTYPE html>
    <html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://code.jquery.com/mobile/1.4.5/jquery.mobile-1.4.5.min.css">
    <script src="https://code.jquery.com/jquery-1.11.3.min.js"></script>
    <script src="https://code.jquery.com/mobile/1.4.5/jquery.mobile-1.4.5.min.js"></script>
    </head>
    <body>""")
    inarr=[]
    textarr=[]  
    for uniq in uniqs:
        strt=copy.copy(start[np.where(data['name_'+language][start]==uniq)[0]])
        a=np.argsort(-data['parent'][strt])
        strt=strt[a]
        
        f.write("""
          <div data-role="header">
            <h1>"""+uniq+"""</h1>""")
        if uniq==uniqs[0]:
            f.write("""
            <button id="collapsible1">"""+bt1+"""</button>
            <button id="collapsible2">"""+bt2+"""</button>
            """)
        f.write("""
          </div>
            <div data-role="main" class="ui-content">
    
        """)
           
            
        n=1
        level=1
        
        for i in strt:
            try:
                print(data['id'][i],data['name_'+language][i],data['id'][i],data['text_'+language][i])
                f.write('''
                <div data-role="collapsible" class="collapsible">''')
                f.write('<h1 class="'+str(data['id'][i])+'">'+str(level)+'.'+'</h1>')
                f.write('<p class="'+str(data['id'][i])+'">'+str(level)+'.'+'</p>')
                a=np.where(inarr==data['id'][i])[0]
                if len(a)==0:
                    inarr.append(data['id'][i])
                    textarr.append(data['name_'+language][i]+': '+data['text_'+language][i])
            except:
                print(data['parent'][i],data['name_'+language][i],data['id'][i],data['text_'+language][i])
            n=n+1
            a1=np.where((np.array(data['parent']) == int(data['item'][i])) & (data['type']=='player'))[0]
            n=loop(a1,n,level,f,inarr,textarr,language=language)
            f.write('</div>')
            f.write('')
            
        f.write(''' 
        </div>''')
    f.write('''
    <script>
                
        
        
        
        window.onload = function func(){
        button1=document.getElementById("collapsible1");
        button2=document.getElementById("collapsible2");
        button1.onclick = function(){
            $( ".collapsible" ).collapsible( "expand" );
            }
        button2.onclick = function(){
            $('[data-role="collapsible"]').collapsible( "collapse" );
            }
        }
        
        
              
                inarr='''+str(inarr)+''';
                textarr='''+str(textarr)+''';
                
                for (i = 0; i < inarr.length; i++) {
                
                $('.'+inarr[i]).each(function(index, obj)
                {
                  $(this).text($(this).text()+textarr[i]);
                });
                }
                </script>
                </body>
                </html>''')

In [40]:
import copy

data=pd.read_csv('PathologicSandbox.csv')
data['id']=data['id']-min(data['id'])+5

start=np.where((data['type']=='character') & (data['name_ru'].isnull()))[0]
data['name_en'].iloc[start]='Person'
data['name_ru'].iloc[start]='Человек'
data['name_br'].iloc[start]='Pessoa'
data['name_kr'].iloc[start]='사람'


name=copy.copy(data['name_en'])
a=np.where(name=="Aglaya's Reflection")[0]
name[a]="Aglaya Lilich"
a=np.where(name=="Alexander's Reflection")[0]
name[a]="Alexander Saburov"
a=np.where(name=="Sahba-ötün's Reflection")[0]
name[a]="Aspity"
a=np.where(name=="Big Vlad's Reflection")[0]
name[a]="Big Vlad"
a=np.where(name=="Clara")[0]
name[a]="Changeling"
a=np.where(name=="Georgiy's Reflection")[0]
name[a]="Georgiy Kain"
a=np.where(name=="Katerina's Reflection")[0]
name[a]="Katerina Saburova"
a=np.where(name=="Lara's Reflection")[0]
name[a]="Lara Ravel"
a=np.where(name=="Oyun's Reflection")[0]
name[a]="Foreman Oyun"

todo=["Aglaya Lilich","Alexander Block","Alexander Saburov","Andrey Stamatin","Aspity","Bachelor","Bad Grief","Big Vlad","Capella","Changeling","Eva Yan","Fellow Traveler","Foreman Oyun","Georgiy Kain","Grace","Isidor Burakh","Katerina Saburova","Khan","Lara Ravel","Maria Kaina","Mark Immortell","Murky","Nara","Peter Stamatin","Rat Prophet","Stakh Rubin","Sticky","Taya Tycheek","Victor Kain","Young Vlad","Yulia Lyuricheva","Var","Person","Anna Angel","Notkin","Wonder Bull","Tumbler Human","Artemy Burakh","???","Executor"]

covered=[]
for i in todo:
    a=np.where(name==i)[0]
    covered.extend(a)
ids=np.arange(len(name))
ids=np.setdiff1d(ids,covered)
name[ids]='Other'


start=np.where(data['text_br'].isnull())[0]
data['text_br'].iloc[start]='nan'
start=np.where(data['text_kr'].isnull())[0]
data['text_kr'].iloc[start]='nan'


start=np.where((data['parent']<=0) & (data['type']=='character'))[0]
ns=np.unique(name[start])

rn=np.where((data['type']=='character') & (data['name_en']=='Person'))[0]
data['name_en'].iloc[rn]=data['authorname'].iloc[rn]
data['name_ru'].iloc[rn]=data['authorname'].iloc[rn]
data['name_br'].iloc[rn]=data['authorname'].iloc[rn]
data['name_kr'].iloc[rn]=data['authorname'].iloc[rn]

lan=['en','ru','br','kr']
lan=['kr']
for l in lan:
    for nnn in ns:
        start=np.where((data['parent']<=0) & (data['type']=='character') & (name==nnn))[0]
        main(start,name=nnn,language=l)


#start=np.where((data['parent']==-65) & (data['parent']<0) & (data['type']=='character'))[0]




516208 ??? 516208 모든 이름, 내가 찾아 가니, 기뻐하며 하나씩 데려갈까, 온전히 데려갈까. 네 가락을 따라 하나씩 죽일까?
916150 ??? 916150 말, 적어. 목구멍, 없어. 어려워.
999705 ??? 999705 나는 일어선다. 누구도 나를 막지 못하리. 너도 나를 막지 못하리. 내가 모두를 데려가리니.
1028449 ??? 1028449 뭐 하러 서두르지? 마을 회관에서 뭘 찾기를 기대했나? 모래 역병 대책 위원회? 마을의 지도자들? 오늘부터 나, 모래 역병, 모래 페스트, 대지에서 나온 살점이 이 마을을 지배하리라.
1059207 ??? 1059207 질문?
1069569 ??? 1069569 엠셴.
1380214 ??? 1380214 ...겁먹었나? 걱정 마, 애들이 어떤 모습일지 짐작하고 있었을 뿐이니까. 다들 더할 나위 없이 멀쩡해... 지금 당장은. 하지만 이미 모래시계는 흐르기 시작했어. 아이들이야말로 가장 <맛있는> 조각이야... 최고로 흥미로운 한 입.
1387125 ??? 1387125 아이를 구했다고 생각하나? 거짓이야! 죽음은 잠시 쉬어갈 뿐. 언제까지나 나를 막을 수는 없으리. 네게 더 이상 낫게 하는 물건이 없을 때, 다시, 또 다시, 돌아와 주지. 얼마 남지 않았어.
625160 아글라야 릴리히 625160 긴 대화가 될 것이다. 이 대화에만 완전히 집중하도록. 진실을 말할 필요는 없지만, 거짓에 대한 책임을 져야 할 것이다. 준비되었나?
763989 아글라야 릴리히 763989 ...사망자 약 7천 명.
798435 아글라야 릴리히 798435 안 된다.
943719 아글라야 릴리히 943719 만능약 재료 확보에는 진전이 있었나, 부라흐? 황소들을 찾아냈나? 우리에겐 더 이상 기다릴 시간이 없다.
1749553 아글라야 릴리히 1749553 "격언은 이야기, 멀리서 바라본 이야기. 듣는 것은 그대의 괴로움, 그대가 느끼는 것은 끓어오르는 무심함."
1059210 아글라야 릴리히 1059210 죽음의 탑

413984 그리프 413984 몇 시간 기다렸다 스텝으로 가서 철로를 따라가... 바깥 세상으로 펼처지는 길다란 선. 네 비유적인 선이 아닌 진짜 선. 거기서 기다리고 있을게.
1467428 그리프 1467428 몇 시간 기다렸다 스텝으로 가서 철로를 따라가... 바깥 세상으로 펼쳐지는 기다란 선. 네 비유적인 선이 아닌 진짜 선. 거기서 기다리고 있을게.
1535108 그리프 1535108 ...하지만 그들 또한 누군가에 의해 조종당하고 있어. <그런> 일을 위해서는 어떤 이빨이 필요할지 상상도 가지 않는군.
1643811 그리프 1643811 허리 좀 펴고 다닐래? 너 인간이 아니라 물음표처럼 보이거든.
1643813 그리프 1643813 옛날 옛적에, 포위당한 마을 하나가 있었습니다. 사람들은 굶주리고 있었습니다. 누군가가 말했습니다, "이제 전부 끝이야. 천사가 보여." 다른 사람이 대답했습니다, "천사를 먹을 수 있을까?"
116091 늙은 블라드 116091 익숙한 얼굴이군... 자네 아버지가 젊었던 시절의 모습이 아직도 생생해... 그래... 내가 이시도어보다 15년 늦게 태어났지. 아주 괜찮은 사람이었어... 쉽게 겁먹지 않는...
116131 늙은 블라드 116131 그만 돌아다니게, 아르테미... 떠날 필요는 없어. 자네를 쇠사슬로 묶어두고 싶지는 않지만, 내 말을 듣게... 바깥 세상에서 자네는... 일을 망치고 다닐 뿐이야.
116154 늙은 블라드 116154 부라흐 도련님, 가만히 있질 못하는군... 아니면 마음이 바뀌었나?
258905 늙은 블라드 258905 전염병이라는군, 부라흐... 역병이라. 5년 전 내 일꾼 300명을 하루 만에 쓸어간 그 모래 역병이라면 상황이 심각해... 심각하고 말고, 아르테미. 그 때는 자네 아버지가 우리를 구했다만... 지금은... 지금은 자네가 필요하구나... 내 가까이. 이렇게 말하면 알아듣겠지?
771506 늙은 블라드 771506 아, 부라흐... 자네는 아직 아내가 없겠지... 그렇지? 


1804205 동반자 1804205 안녕, 동반자! 딱 맞춰서 왔네!
841506 오윤의 그림자 841506 9년 전, 제 주인님께서는 슬프게도 도살장의 작업반장이 되셨습니다. 올김스키는 일족을 자신의 의지대로 강제하기 위해, 그들이 노동이라는 쓰디쓴 멍에에 대항해 반란을 일으키는 것을 방지하기 위해, 신뢰할 수 있는 일족 사람을 원했습니다.
763907 작업반장 오윤 763907 우넨테, 아하르. 드디어 왔군.
916107 작업반장 오윤 916107 올롱고로, 도살장으로 향할 때가 왔다.
916109 작업반장 오윤 916109 돌아왔군. 도살장에 들어갔다 왔군.
923332 작업반장 오윤 923332 왔는가.
1000412 작업반장 오윤 1000412 네가 무언가를 이해했다면, 그것으로 충분하다.
1559106 작업반장 오윤 1559106 그들이 네게 무엇을 원하지? 행동. 그래, 행동. 하지만 무슨 행동?
1608539 작업반장 오윤 1608539 잔인하군, 형제. 나를 죽였어야 했다.
1660418 작업반장 오윤 1660418 가을의 스텝은 아름답다... 그리고 가을은 진실의 계절. 약속을 지키는 계절. 우리가 지은 죄와, 우리가 이룬 업적의 대가를 치르는 계절. 우리의 실수를 정화하는 계절. 좋은 계절이다.
1659930 작업반장 오윤 1659930 내 상처를 보지 마라, 혈족이여. 가서 행동으로 증명해라.
1707875 작업반장 오윤 1707875 무엇을 원하는가, 혈족이여? 말해라. 너는 왜 대지 위에 살아가는가?
1686268 작업반장 오윤 1686268 잔인하군, 형제. 나를 죽였어야 했다.
1759705 작업반장 오윤 1759705 위대한 행동을 했군, 형제. 장로들은 "톰 나이마"라고 불렀겠지. 나는 "행동"이라고 부르겠다. 전부 혼자서 이뤄내다니.
1759722 작업반장 오윤 1759722 힘의 길을 골랐군, 형제. 네 선택을 존중한다.
1660482 작업반장 오윤 1660482 모든 피는 올롱고에서 스며나온다. 도살장으로부터. 언제나. 피

1832529 마크 이모텔 1832529 내게 질문하고 실어 입이 근질거리지?
116966 미쉬카 116966 저리 가. 나는 다 봤어. 아무도 나를 못 봤어. 나는 봤어.
117013 미쉬카 117013 왜 왔어? 필요 없어. 다들 아저씨 필요 없대. 어차피 이젠 걔랑 친구 아냐.
117016 미쉬카 117016 그래서?
478190 미쉬카 478190 왜 아저씨를 사랑해야 해? 사랑할 필요 없어. 아무 때나 그만 사랑할 수 있는데.
661205 미쉬카 661205 아저씨네 집, 더러워.
665885 미쉬카 665885 몰라... 아저씨 딸 같은 것처럼 해줄 수도 있는데.
665903 미쉬카 665903 내 친구가 아저씨 만날 준비가 됐대. 그렇게 말했어. 그렇게.
710630 미쉬카 710630 맛난 거 가져왔어?
710650 미쉬카 710650 내 친구가 알고 싶대. 왜 아저씨는 맨날 내 친구에 대해 물어보고 다녀? 아저씨도 친구 하고 싶어?
710678 미쉬카 710678 친구가 거의... 거의 다 왔어. 근데 왜 안 나올까. 내가 가서... 불러야 하나 봐.
713206 미쉬카 713206 내 친구가 알고 싶대. 왜 아저씨는 맨날 내 친구에 대해 물어보고 다녀? 아저씨도 친구 하고 싶어?
713834 미쉬카 713834 또 아저씨 아빠 때문에 스피카랑 싸웠어! 다시는 화해 못하면 어쩌지... 너무 걱정돼.
1004405 미쉬카 1004405 내가 친구를 만나러 가서 슬퍼? 슬퍼하지 마! 다 이유가 있어. 아저씨한테 세 가지를 말해 주기 위해 죽었어. 아저씨가 알아야 할 것 같아.
1018305 미쉬카 1018305 여기가 우리 새 집이야?
1096008 미쉬카 1096008 여기? 여기... 여기.
1096061 미쉬카 1096061 응. 맞는 풀이야. 마법의 풀.
1482927 미쉬카 1482927 ...나한테 화났어?
1608557 미쉬카 1608557 예쁘지? 진짜 집 같이.
1772476 미쉬카 1772476 냄새 맡고 알았어.
1004436 

116868 리카 116868 곱추 치고는 꽤 강한걸! 아저씨, 큰 실수 했어! 셋 다 죽어 버렸는걸. 아저씨가 죽이는 거, 내가 전부 봤어. 큰일 났네!
116901 리카 116901 아저씨, 좋아 보이네! 마을이 환영해 주지 않았지? 걱정 마. 스텝은 넓어서 누구나 숨겨 주니까. 왜 왔어?
116938 리카 116938 멈춰! 놀아!
116948 리카 116948 칼 든 놈은 가죽 세공사 키리크였어. 원래부터 머리가 좀 이상한 놈이었으니까 죽었다고 별로 슬프진 않지만.
116927 리카 116927 싫어, 싫어! 용서해 줘! 우린 친구잖아! 내가 아저씨를 도와줬잖아! 아저씨는 나한테 빚을 졌잖아! 아저씨한테는 아무런 의미도 없는 거야? 나를 놓아줘! 아저씨한테 쓸모 있을지도 몰라!
186111 리카 186111 혼자서 생각하는 방법을 모르는 사람도 필요 없어. "그 놈을 잡아" 라고 그들이 명령하니까 아저씨는 바로 나를 물러 왔잖아. 나를 놓아줘, 아저씨. 착하지?
830325 마을의 그림자 830325 저희 목소리를 듣고 찾아오시다니, 참으로 훌륭한 의사 선생님이시군요.
830344 마을의 그림자 830344 이 집에는 마을의 유일한 바퀴 만드는 장인이 살고 있었습니다. 저희들의 공장은 바퀴를 제조하지 않습니다. 대신 황소들을 연골과 뼈와 피로 분해합니다.
834419 마을의 그림자 834419 이 집에는 물 운반꾼들이 살고 있었습니다. 그들은 매주 목요일마다 물통을 마을로 실어왔습니다. 마을이 역병에 휩싸여 격리당하기 직전, 포르투나는 그들을 우리 품으로 돌려보냈습니다.
834430 마을의 그림자 834430 이 집에는 두 가족이 살고 있었고, 둘 다 안드레이 스타마틴에게 마지막으로 남은 가보를 팔아넘겼습니다. 술집 주인은 금을 사들이고 있습니다.
1672307 머리가 짧은 마을 사람 1672307 그래서? 그 여자를 처형했어?
1672314 머리가 짧은 마을 사람 1672314 여기... 공공 보호소 맞지? "산 자들의 집"? 집이 불타 버렸거나, 격

698036 아이 698036 벌써 2주나 늦었어. 어떤 멋진 소포가 도착할지 기대하고 있었는데...
698043 아이 698043 달리는 기차. 최고로 빠른 기차. 재빠른. 날렵한. 어... 즉각적인. 으음... 이름을 잘못 골랐나 봐...
1063937 아이 1063937 마을 한쪽에서 쇠로 된 껍질이 자라고 있어. 철로 된 창고 건물들이 땅을 파고들고 자라고, 큰 기차역에서 뻗어 나온 선로도 땅을 파고들고 자라거든.
1063966 아이 1063966 ...철도가 스텝으로 스며들어. 철이 스텝으로 스며들어. 기다란 것들이 스며들어. 녹슨 바퀴들도. 풀숲 속을 기차 바퀴가 파고들었어.
1063972 아이 1063972 ...나는 철이 싫어. 사악하고, 낯설고, 녹슬었으니까.
1251682 아이 1251682 신선한 견과 있어? 희귀한 견과랑 교환해 줄게.
1439415 아이굴 1439415 소는 잘 지내나, 하탕게?
1563309 아이사 1563309 안녕, 의사 선생. 악수해도 될까?
1169913 안내인 1169913 떠나시렵니까? 마을에서 당신을 기다리는 사람들이 있지 않습니까?
118067 앗사 118067 그 재수없는 놈 아직 거기 있어요? 손가방 든 그 사람?
1205625 앗사 1205625 조심해요. 그러다가 여주인들이 깨어나면 어쩌려고요? 빅토리아는 몰라도, 니나가 깨어나면...
595114 야크멘 595114 이 철로는 탯줄이야. 우리를 바깥 세상과 이어 주는 유일한 존재.
1118947 언니 1118947 엄마가 끔찍한 병에 걸렸어요, 의사 선생님. 동생을 지킬 수 있게 약 주시면 안 될까요? 동생은 몸이 약하거든요... 이번 9월은 정말 끔찍해요...
1498509 연결 고리 1498509 미친 질문만 해대는 여자야! <무슨> 숫자? 내 근무 일정... 내가 죽이고 싶은 사람이 누군지... 왜 나처럼 평범한 사람에게 묻는 거야? 나는 생산적인 사회의 일원인데다, 지금까지 선하게 살아오면서 아무도 다치게 한 적 없는데...
1498511 

710605 CharactersCrowdPassers_speech_1 710605 이시도어를 죽인 범인은 찾았어?
739329 CharactersCrowdPassers_speech_1 739329 양심 때문에라도 당신과 거래하고 싶지만, 도둑이나 살인자와는 아무도 거래하지 않아. 물물교환은 신뢰에 기반한 거래야.
753498 CharactersCrowdPassers_speech_1 753498 일하는 중! 일하는 중이야! 곧 끝나. 아저씨도 마음에 들어 할 거야.
753500 CharactersCrowdPassers_speech_1 753500 스텝 놈들이 소를 끌고 설도 지구를 지나가고 있어. 잔인함을 기리는 짐승들 같으니...
943705 CharactersCrowdPassers_speech_1 943705 심문관이 체포당했대. 아직 체포당하지 않았다면 곧 잡힐 거래. 다행이라고 생각해. 군대가 우리를 지켜줄 거야. 군대는 항상 코미사르를 증오했으니까, 그 여자를 잡아서 기뻐하고 있겠지. 
977305 CharactersCrowdPassers_speech_1 977305 닥터 단코프스키의 명령에 따라, 허가된 사람 외에는 감염된 지구에 출입 금지... 당신이 그 보라흐인지 뭔지 하는 사람인가?
1251605 CharactersCrowdPassers_speech_1 1251605 이 물건을 며칠 동안 지니고 다녔어. 물론 쉽지 않았지. 이 물건과 함께 공포, 내 슬픔, 내 분노도 같이 나눌게. 힘들었지만 이젠 약간 수월해진 느낌이 들어. 아주 약간. 내 짐을 덜어줘서 고마워.
1251615 CharactersCrowdPassers_speech_1 1251615 광장에서 아저씨를 기다리는 애들이 있어. 바위 정원에 있는 다리 광장. 어떻게 가는지 알아?
1311832 CharactersCrowdPassers_speech_1 1311832 오늘 아침에 내 작은 생쥐들에게 식료품 심부름을 시켰는데, 아직도 돌아오지 않았어! 식량이 기적적으로 저절로 구매되리라고 생

1324639 Template_Event_Nazareth_Althruism_speech 1324639 어머니에게 자기희생에 대해 묻고 싶어. 어머니는 우리가 마을을 범람할 준비를 하는 일족을 위해 모든 것을 희생해야만 한다고 말씀하셨어... 나는 내 내장을 포함해 내가 가진 모든 것을 바쳤고, 이제는 죽어가고 있어. 나는 옳은 결정을 내렸을까?
1324608 Template_Event_Nazareth_First_speech 1324608 사바는 우리를 가르칠 거야. 사바는 길을 아는 자, 대지와 말하는 자. 어머니 보도는 사바를 통해 우리에게 말해.
1324652 Template_Event_Nazareth_Hive_speech 1324652 어머니에게 어떻게 자신을 부정할 수 있는지 묻고 싶어. 내 몸이 도저히 자신을 잊으려 들지 않아. 몸이 존재하는 한은 자신에 대한 사랑이 멈추지 않아.
1324629 Template_Event_Nazareth_Incest_speech 1324629 사바에게 우리 형제들과 자매들이 결혼하는 것이 허락될지 묻고 싶어. 일족은... 일족이잖아. 우리는 모두 보도의 아이들인데, 몇 명 남지 않았으니까...
1324622 Template_Event_Nazareth_Killing_speech 1324622 어머니에게 질문하고 싶어. 모든 것이 끝나기 전까지 시간이 얼마 남지 않았어. 때가 되면, 나를 친구라고 불러 주고, 나를 신뢰해 준 자들을 살려 줘도 괜찮을까? 그들을 죽이지 않아도 괜찮을까?
1324659 Template_Event_Nazareth_Language_speech 1324659 어머니에게 우리의 언어에 대해 묻고 싶어. 마을 사람들의 말은 어려워. 우리의 정신을 느리게 만들어. 어떻게 거절해야 하지?
1324645 Template_Event_Nazareth_Nomina_speech 1324645 내 이름을 버리고 싶어. 내 가족들도 이름을 버리고 싶어 해. 에트세그 에이, 아브가이, 바사간, 아크샨. 왜 이름이 필요

1745932 마법의 황소 1745932 ....
1745934 마법의 황소 1745934 우헤르네이 에리드 바이나 기이?
116344 젊은 블라드 116344 낯익은 얼굴이군요. 전에 뵌 적이 있던가요? 무슨 일로 오셨죠?
116375 젊은 블라드 116375 한 잔 하시겠습니까?
267230 젊은 블라드 267230 굴욕적이군요... 아버지께서는 의도적으로 제게 올김스키 가문을 가능한 한 부정적으로 보이게 하는 일들을 맡기셨습니다. 기금에 충분한 양의 식량과 의약품을 제공하는 일은 누가 봐도 불가능하지 않습니까. 제 노력은 전부 허사입니다!
267289 젊은 블라드 267289 아, 부라흐 씨. 때맞춰 오셨군요. 이 열쇠는 마땅히 당신의 것입니다. 당신 아버지께서 작업실로 사용하던 공장 건물 3번의 열쇠입니다. 받아 주십시오. 단지... 저희가 나눈 대화의 내용을 공공연히 퍼뜨리고 다니지 말아 주셨으면 합니다.
439432 젊은 블라드 439432 부라흐 씨.
439434 젊은 블라드 439434 그들의 말을 믿습니까? 부라흐 씨, 대답하기 전에 부디 심혈을 기울여 생각해 보기를 간청합니다. 당신의 대답에 수천 명의 목숨이 달려 있을지도 모릅니다.
522448 젊은 블라드 522448 다시 만나서 반갑습니다, 부라흐 씨.
794179 젊은 블라드 794179 지금은 곤란합니다. 저는 떠나야 합니다.
796310 젊은 블라드 796310 그들에게 아버지를 가만 내버려 두라고 전해 주세요.
796313 젊은 블라드 796313 모든 것이 제자리에.... 부라흐 씨, 모든 것이 제자리에...
1018109 젊은 블라드 1018109 어머니께서 살아 계셨다면 제 계획을 지지하셨을 겁니다. 어미니께서는 사람들을 사랑하셨습니다. 저는 사랑하지 않습니다. 그럼에도 불구하고, 저는 존경심을 가지고 사람을 대하면 그는 나중에 백 배로 은혜를 갚을 것이라고 믿습니다. 경제 성장은 인간의 존엄성에 기반합니다. 그리고 이 마을에서 "존엄성"이라는 단어는 듣지 못하게 된 지 오래

In [43]:
import copy

data=pd.read_csv('PathologicSandbox.csv')
data['id']=data['id']-min(data['id'])+5

start=np.where((data['type']=='character') & (data['name_ru'].isnull()))[0]
data['name_en'].iloc[start]='Person'
data['name_ru'].iloc[start]='Человек'
data['name_br'].iloc[start]='Pessoa'
data['name_kr'].iloc[start]='사람'


name=copy.copy(data['name_en'])
a=np.where(name=="Aglaya's Reflection")[0]
name[a]="Aglaya Lilich"
a=np.where(name=="Alexander's Reflection")[0]
name[a]="Alexander Saburov"
a=np.where(name=="Sahba-ötün's Reflection")[0]
name[a]="Aspity"
a=np.where(name=="Big Vlad's Reflection")[0]
name[a]="Big Vlad"
a=np.where(name=="Clara")[0]
name[a]="Changeling"
a=np.where(name=="Georgiy's Reflection")[0]
name[a]="Georgiy Kain"
a=np.where(name=="Katerina's Reflection")[0]
name[a]="Katerina Saburova"
a=np.where(name=="Lara's Reflection")[0]
name[a]="Lara Ravel"
a=np.where(name=="Oyun's Reflection")[0]
name[a]="Foreman Oyun"

todo=["Aglaya Lilich","Alexander Block","Alexander Saburov","Andrey Stamatin","Aspity","Bachelor","Bad Grief","Big Vlad","Capella","Changeling","Eva Yan","Fellow Traveler","Foreman Oyun","Georgiy Kain","Grace","Isidor Burakh","Katerina Saburova","Khan","Lara Ravel","Maria Kaina","Mark Immortell","Murky","Nara","Peter Stamatin","Rat Prophet","Stakh Rubin","Sticky","Taya Tycheek","Victor Kain","Young Vlad","Yulia Lyuricheva","Var","Person","Anna Angel","Notkin","Wonder Bull","Tumbler Human","Artemy Burakh","???","Executor"]

covered=[]
for i in todo:
    a=np.where(name==i)[0]
    covered.extend(a)
ids=np.arange(len(name))
ids=np.setdiff1d(ids,covered)
name[ids]='Other'


start=np.where(data['text_br'].isnull())[0]
data['text_br'].iloc[start]='nan'
start=np.where(data['text_kr'].isnull())[0]
data['text_kr'].iloc[start]='nan'


start=np.where((data['parent']<=0) & (data['type']=='character'))[0]
ns=np.unique(name[start])

#rn=np.where((data['type']=='character') & (data['name_en']=='Person'))[0]
#data['name_en'].iloc[rn]=data['authorname'].iloc[rn]
#data['name_ru'].iloc[rn]=data['authorname'].iloc[rn]
#data['name_br'].iloc[rn]=data['authorname'].iloc[rn]
#data['name_kr'].iloc[rn]=data['authorname'].iloc[rn]

from astropy.table import Table
t=Table({'game':[""],"language":[""],"protagonist":[""],'protorder':[1],"name":[""],"path":[""],"order":[0],
         'lanorder':[0],'gameorder':[5]})

order=np.array([41,5,6,13,16,18,35,29,3,21,11,10,4,17,40,32,30,8,27,34,14,26,19,7,31,24,36,25,42,43,15,33,22,23,28,38,37,9,39,12,20])-1

other=['Other','Другие','Outros','기타']
person=['Shared Dialogue','Общие диалоги','Diálogo Compartilhado','공통 대화']

lan=['en','ru','br','kr']
lans=['English','Русский','Português brasileiro','한국어']
lanorder=[0,1,3,3]
for l,ls,oth,per,lanor in zip(lan,lans,other,person,lanorder):
    for nnn,o in zip(ns,order):
        start=np.where((data['parent']<=0) & (data['type']=='character') & (name==nnn))[0]
        if nnn=='???':
            f = 'html2_'+l+'/'+''.join('unknown')+'.html'
        else:
            f = 'html2_'+l+'/'+''.join(nnn.split(' '))+'.html'
        #print(f,data['name_'+l][start[0]])
        a1=np.where((np.array(data['parent']) == int(data['item'][start[0]])) & (data['type']=='player'))[0]
        
        if nnn=='Other':
            nam=oth
        elif nnn=='Person':
            nam=per
        else:
            nam=data['name_'+l][start[0]]
        t.add_row({'game':'Pathologic 2', 'language':ls,"protagonist":data['name_'+l][a1[0]],
                   "protorder":1,"name":nam,"path":f,"order":o,'gameorder':5,'lanorder':lanor})
        
        #main(start,name=nnn,language=l)
t

#start=np.where((data['parent']==-65) & (data['parent']<0) & (data['type']=='character'))[0]




game,language,protagonist,protorder,name,path,order,lanorder,gameorder
str12,str20,str8,int64,str21,str30,int64,int64,int64
,,,1,,,0,0,5
Pathologic 2,English,Haruspex,1,???,html2_en/unknown.html,40,0,5
Pathologic 2,English,Haruspex,1,Aglaya Lilich,html2_en/AglayaLilich.html,4,0,5
Pathologic 2,English,Haruspex,1,Alexander Block,html2_en/AlexanderBlock.html,5,0,5
Pathologic 2,English,Haruspex,1,Alexander Saburov,html2_en/AlexanderSaburov.html,12,0,5
Pathologic 2,English,Haruspex,1,Andrey Stamatin,html2_en/AndreyStamatin.html,15,0,5
Pathologic 2,English,Haruspex,1,Anna Angel,html2_en/AnnaAngel.html,17,0,5
Pathologic 2,English,Haruspex,1,Artemy Burakh,html2_en/ArtemyBurakh.html,34,0,5
Pathologic 2,English,Haruspex,1,Aspity,html2_en/Aspity.html,28,0,5


In [44]:
import copy

ver=['MarbleNest','PathologicPlagueIntro','PathologicHaruspexIntro']

m=["Marble's Nest","У мраморного гнезда","Ninho marmóreo","대리석 둥지"]
pi=["Day 12 intro","День 12 вступление","Dia 12 introdução","프롤로그: 12일차"]
pt=["Train intro","Поезд вступление","Trem introdução","프롤로그: 열차"]
protorder=[0,1,1]

group=[m,pi,pt]
order=[0,0,1]
for x,gr,o,prot in zip(ver,group,order,protorder):
    data=pd.read_csv(x+'.csv')
    start=np.where((data['type']=='character') & (data['name_ru'].isnull()))[0]
    data['name_en'][start]='Person'
    data['name_br'][start]='Pessoa'
    data['name_ru'][start]='Человек'
    data['name_kr'][start]='사람'
    data['id']=(data['id']-np.min(data['id'])).astype(int)
    
    rn=np.where((data['type']=='character') & (data['name_en']=='Person'))[0]
    data['name_en'][rn]=data['authorname'][rn]
    data['name_ru'][rn]=data['authorname'][rn]
    data['name_br'][rn]=data['authorname'][rn]
    data['name_kr'][rn]=data['authorname'][rn]
    
    
    start=np.where((data['parent']<=0) & (data['type']=='character'))[0]
    lan=['en','ru','br','kr']
    for l,ls,g,lanor in zip(lan,lans,gr,lanorder):
        start=np.where((data['parent']<=0) & (data['type']=='character'))[0]
        main(start,name=x,language=l)
        
        a1=np.where((np.array(data['parent']) == int(data['item'][start[0]])) & (data['type']=='player'))[0]
        f = 'html2_'+l+'/'+''.join(x)+'.html'
        
        t.add_row({'game':'Pathologic 2', 'language':ls,"protagonist":data['name_'+l][a1[0]],
                   "protorder":prot,"name":g,"path":f,"order":o,'gameorder':5,'lanorder':lanor})

a=np.argsort(t['order'])
t=t[a]
t

#start=np.where((data['parent']==-65) & (data['parent']<0) & (data['type']=='character'))[0]




30 ??? 30 You are a brave man, doctor. I'm glad to see you've finally mustered up the courage to do it. Meaning it's probably time to say goodbye, right?
94604 Aspity 94604 Hello, doctor. What did you dream of today?
94679 Aspity 94679 ...You are far from cowardly, doctor. Mulish though. Your head is stuffed with science; your eyes, sewn shut.You will walk this path time and again, until every last bit of your resolve wears off. And who are you without it? Who would need you?
94686 Aspity 94686 See? There's nothing terrible about it. I died properly.
188205 Aspity 188205 Most importantly, don't be scared. Go in without fear. Concentrate on yourself.
188211 Aspity 188211 You're too early. Not everyone's here yet.
1694 Believer 1694 I came here before you. But you may go in first. I need to pluck up my courage.
1437 Blacky 1437 It's not the sand pest. Don't tell 'em beakheads!He'll get better. I know it.
79443 Blacky 79443 You are late. A man died.Allow me to draw your attention to the f

90153 Marat Ranin 90153 Your stubbornness will be your undoing... and your main regret. You'll find yourself in our shoes sooner than you think.
90184 Marat Ranin 90184 Get a grip! Now is not the time to be afraid.
90191 Marat Ranin 90191 You're back. So it's true? That you're losing your grip and giving up?
1860 Merchant 1860 No money. Barter only.
1918 Mourner 1918 Did you come to pay respects?
435 Odongh 435 [The creature is breathing heavily. A deep and dirty wound is gaping across its belly. No one's bothered to dress it. To operate on him, one would require clean bandages and painkillers. And you'll have to move fast.]
459 Odongh 459 Enemy. Spoiled everything.
2834 Orderly 2834 Oh, and one more thing, sir. The anti-epidemic committee at the Crucible keeps on working, as far as I know.But we're receiving weird contradictory orders from them. Should we still follow them?
2808 Orderly 2808 I'm standing my watch, doctor. As per your instructions.My job is to educate the local populac

122522 Shrew 122522 ...Get better soon, doctor. And I'm so sorry for you.
122526 Shrew 122526 So here's how it's gonna be with you, huh! Well, in that case, you're no friend of mine!
122541 Shrew 122541 It's not like I don't feel sorry for you. Perhaps I'll even deign to help you after all. But I'll never ever ask you for help again, no sir!
127600 Shrew 127600 You've almost come to. See? We've been here all along.
2855 Siskin 2855 You must be thinking that you can't leave this place now. But don't let it get to you.
120527 Sleepy Head 120527 Just... dontcha worry, 'kay? 'Twill be over soon.
120500 Sleepy Head 120500 Shrew's got the keys. Not me. I don't need to go in there anyway. One can catch 'em from over here just fine.
127606 Sleepy Head 127606 Look y'all, he's waking up... What if he attacks us? I'd say we make a run for it!
1722 Soldier 1722 Nothing to see here, doctor. It's all over.A shame you weren't here earlier... but there's nothing to be done now.
2974 Soldier 2974 The b

180776 SOURCE_Martha2_WanderArea_talking_412_speech 180776 Я проиграла душу.
180783 SOURCE_Martha2_WanderArea_talking_412_speech 180783 А все-таки вам удалось одолеть чуму... если бы не вы, лежать бы нам всем в сырой земле. Вы только скажите - теперь-то точно всё позади? Зараза уже не вернётся?
180901 SOURCE_Martha2_WanderArea_talking_412_speech 180901 Солдат приставили к аптеке - той, что возле моста. Не пускают туда никого. Значит, скоро откроют? 
180908 SOURCE_Martha2_WanderArea_talking_412_speech 180908 Что с нами будет?
2149 The Lil' One 2149 Все... я всех отнёс. Больше не могу.  
103114 TragedianClerk_1_НОВЫЙ_speech 103114 Убедительно просим посторонних немедленно удалиться! Исключительно много работы.
121844 Белка 121844 ...Выздоравливай скорей, доктор. Так жалко тебя.
121800 Белка 121800 Ну? Собрал?
122522 Белка 122522 ...Выздоравливай скорей, доктор. Так жалко тебя.
122526 Белка 122526 Ах вот ты, значит, как! Я с тобой тогда больше не вожусь.
122541 Белка 122541 Может, я тебя 

85563 Спичка 85563 Страшно-то как... Но ты ничего, доктор, не поддавайся. Мы с тобой еще повоюем, да? Правда?
116165 Спичка 116165 У меня к тебе важная просьба. Не слушай Белку. И орехов ей ни за что не давай. Это ловушка. Понял?
116169 Спичка 116169 Ну что Черныш?
117458 Спичка 117458 Только не говори Белке, что я тебя предупредил, ладно? Она вообще-то хороший друг...
117463 Спичка 117463 Ты правда хочешь умереть, да? Тебе надоело с нами?
117482 Спичка 117482 Что, доктор, тяжело тебе?.. Я на тебя не в обиде. Ты просто не разобрался, что к чему... Взрослые часто так.
1860 Торговец 1860 Не надо денег. Только вещи.
417 Травяная невеста 417 Ты врач. Мы тебя ждали.
2941 Травяная невеста 2941 Теперь всё будет хорошо. Мы тебе благодарны. 
2938 Травяная невеста 2938 Нужно, чтобы он жил. Пусть страдает. Лишь бы не умер сегодня.
2954 Травяная невеста 2954 Ну вот. Теперь никто не спасется...
84300 Травяная невеста 84300 ...Ты слышишь?
84318 Травяная невеста 84318 Что бы тебе ни говорили - не вер

...

117458 Palitinho 117458 Só não fala pra Esquilinha que eu te avisei, beleza? Somos bons amigos, sabe?...
117463 Palitinho 117463 Você realmente, de coração, deseja morrer? Você se cansou da gente?
117482 Palitinho 117482 Você deve estar se sentindo muito mal, não é mesmo?... Tá tudo bem. Eu te perdoo. Você só acabou se confundindo... Adultos sempre fazem isso.
476 Pega 476 Atenção, irmãos e irmãs. Um homem-cobra surge. Moga será seu nome.
212920 Pega 212920 O casamento foi concretizado. Isto é para ti. Não podes recusar.
212928 Pega 212928 Vá. Estás a perturbar o luto dela.
2149 Pequenino 2149 Está feito... Levei todos eles. Estou no meu limite.
1918 Pessoa de luto 1918 O senhor também está de luto?
2855 Pintagol 2855 Você deve estar pensando que não pode sair deste lugar agora. Mas não deixe que isso lhe suba à cabeça.
571 Porteiro 571 O Juiz perdeu o juízo, Bacharel. Ele nega o óbvio. Alega que a epidemia ainda está aqui...
1880 Relojoeiro 1880 Sempre que meus olhos se deparam com aq

2987 Tragediante 2987 É preciso olhá-la nos olhos. Cara a cara. Tête-à-tête.
3011 Tragediante 3011 Está nas paredes. Ouça com atenção. Observe com atenção. Caso você veja seu rosto, não desvie o olhar.
1212 Vigia 1212 ...
147115 12번 잡역부 147115 12번, 선생님의 명령대로 도착했습니다. 저기 불 옆에 있었는데, 선생님께서는 확실히 살아계시는군요. 누가 서류 내용 따위에 신경 씁니까? 어차피 전 글도 못 읽으니...
147125 12번 잡역부 147125 시작해도 되겠습니까, 선생님?
2712 2번 잡역부 2712 미카일 고바, 2번. 보고드립니다. 있잖아요, 그들이 선생님께서 결딴났다고 했을 때 저는 안 믿었거든요... 이것 봐요! 제가 옳았습니다!
141300 2번 잡역부 141300 미카일 고바, 2번. 명령을 내리십시오, 선생님...
2894 2번 잡역부 2894 미카일 고바, 2번. 명령을 내리십시오, 선생님...
2686 6번 잡역부 2686 그리고리 고르키, 6번. 보고하러 왔습니다, 선생님.
30 ??? 30 드디어 결심한 모양이군. 의사 선생이 용감한 사람이라 다행이야. 그럼, 아마도 작별할 시간이겠지?
2740 ?번 잡역부 2740 팡크라트 린코바르임다, 학사 선생님. 번호가 기억이 안 납니다. 골칫거리가 두 가지 있는데, 어느 쪽이 더 나쁜지 모르겠슴다. 1번, 스텝 사람들. 어제 대성당 뒤에 있던 야영지를 버리고 떠남. 남아 있는 사람들은 누군가의 장례식을 준비하고 있는 것 같슴다. 2번, 법관이 완전히 돌아 버림. 선생님이 내린 방역 조치를 죄다 취소하고... 감염된 사람을 숨겨주고 있는 것 같은데요.
199400 Characters_Gypsies_ОбщийДиалогНаВсехЦыганПослеКолокола_speech 199400 벌써 시작했나 보

2987 비극 배우 2987 시선을 마주해야 합니다. 얼굴을 똑바로 바라보십시오.
3011 비극 배우 3011 벽을 보십시오. 귀를 기울이시고, 주의 깊게 바라보십시오. 그것의 얼굴을 보면, 시선을 돌리지 마십시오.
1860 상인 1860 돈 안 받습니다. 물물교환만 받습니다.
93011 생각에 잠긴 오동그 93011 어린 작물이 곧 올라온다. 바위 꽃.
93017 생각에 잠긴 오동그 93017 공기가 변한다. 들어 봐. 냄새가 변한다. 만져 봐. 사악한 무언가의 조짐이 느껴진다. 바로 여기.
472 샤르 슈부우 472 여긴 네가 있을 곳이 아니야. 결혼식. 너는 못 본다.
476 샤즈가이 476 형제들이여, 뱀 인간이 다가온다. 그를 모가라고 부르자.
212920 샤즈가이 212920 결혼식은 무사히 끝났다. 너를 위한 선물, 거절하지 말아라.
212928 샤즈가이 212928 그녀의 슬픔을 방해하지 말아라, 가.
120527 소냐 120527 그냥... 선생님은 걱정하지 말고 있어. 곧 끝날 거니까, 응?
120500 소냐 120500 열쇠는 벨카가 갖고 있어. 어차피 난 들어갈 필요도 없거든. 여기서도 충분히 잡을 수 있어.
127606 소냐 127606 야, 이것 봐, 선생님이 일어나려고 해... 일어나서 우리를 회초리로 때리면 어쩌지? 도망가자!
103500 소란스러운 남자 103500 역병을 퍼뜨리던 놈을 잡았어... 이놈이 틀림없어.
103509 소란스러운 남자 103509 잘 뒈졌다, 역병 덩어리... 그가 범인이야. 역병을 가져왔다고. 그 말고 다른 누가 있겠어? 스텝 놈들의 둥지도 잿더미로 만들어야 해... 밤에 사람들을 모아, 횃불을 들고 전부 태워 주지!
79425 스피카 79425 무슨 꿈 꾸고 있었어요? 악마들이 선생님을 갈고리로 잡아서 지옥으로 끌고 가려는 것처럼 비명을 지르시던데...
1392 스피카 1392 선생님, 일어나요. 나쁜 일이 일어났어요.
85563 스피카 85563 무서워요... 그래도... 정신 차려야 해요. 알겠죠

430963 Fellow Traveller 430963 We're almost there.
23559 Kin One 23559 Let Mother Boddho caress your step, kholboön. You haven't forgotten your people, have you?
23561 Kin One 23561 Let us trade hearts, kholboön. Let's become kin.
498400 Pack 498400 Each has two eyes and two ears. Most have one nose and one mouth.Riddle me this: If eyes and ears were switched, what would be right? If noses and mouths were switched, what would be fixed?
14500 Survivor 14500 Someone heavy... I hear heavy steps, in the dark. I can't see you. Are you death?
14500 Живая 14500 Тяжелый кто-то... По шагам слышу, тяжелый. В глазах темно. Я тебя не вижу. Смерть моя, что ли, пожаловала?
24206 Исполнитель 24206 Рельсы - это красиво. Они всегда ведут к одной цели, и свернуть с неё невозможно. Разве что только вот так, глупо - потерпеть крушение по пути. Погибнуть, лишь бы только избежать своей судьбы. Глупо. Скучно.
23559 Кровный 23559 Пусть Мать Бодхо греет твои следы, холбоон. Ты не забыл свой народ, так, да?
235

game,language,protagonist,protorder,name,path,order,lanorder,gameorder
str12,str20,str8,int64,str21,str37,int64,int64,int64
,,,1,,,0,0,5
Pathologic 2,한국어,창점술사,1,프롤로그: 12일차,html2_kr/PathologicPlagueIntro.html,0,3,5
Pathologic 2,Português brasileiro,Arúspice,1,Dia 12 introdução,html2_br/PathologicPlagueIntro.html,0,3,5
Pathologic 2,Русский,Гаруспик,1,День 12 вступление,html2_ru/PathologicPlagueIntro.html,0,1,5
Pathologic 2,English,Haruspex,1,Day 12 intro,html2_en/PathologicPlagueIntro.html,0,0,5
Pathologic 2,한국어,학사,0,대리석 둥지,html2_kr/MarbleNest.html,0,3,5
Pathologic 2,Português brasileiro,Bacharel,0,Ninho marmóreo,html2_br/MarbleNest.html,0,3,5
Pathologic 2,Русский,Баклавр,0,У мраморного гнезда,html2_ru/MarbleNest.html,0,1,5
Pathologic 2,English,Bachelor,0,Marble's Nest,html2_en/MarbleNest.html,0,0,5


In [45]:
t[1:].write('p2names.csv',overwrite=True)

In [ ]:
#data=pd.read_csv('PathologicSandbox.csv')
#start=np.where((data['type']=='character') & (data['name_ru'].isnull()))[0]
#data['name_en'][start]='Person'
#data['name_ru'][start]='Человек'

a=np.where((data['type']=='character') & (data['parent']<=0))[0]
print(len(a))
u, indices,n = np.unique(data['name_ru'][a], return_index=True,return_counts=True)
for i in range(len(u)):
    print('"'+u[i]+'"',n[i])